# FRAUD DETECTION DASHBOARD

## Importing libraries

In [38]:
import numpy as np
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dash  # Use Dash instead of JupyterDash
from dash import dcc, html, Input, Output  # Condense all Dash-related imports
import pickle  # For loading saved models

In [43]:
# Initialize the app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Load your dataset

data_credit_card = pd.read_csv('C:/Users/user/Documents/PROJECTS/DATA SCIENCE PROJECTS/CREDIT CARD FRAUD DETECTION/credit.csv') 
model_metrics = pd.read_csv("C:/Users/user/Documents/PROJECTS/DATA SCIENCE PROJECTS/PROJECTS ON GITHUB/model_metrics.csv")  # File to store model evaluations


In [40]:
# App Layout
app.layout = html.Div([
    html.H1("Fraud Detection Dashboard", style={"text-align": "center"}),

    # Section 1: Model Performance Metrics
    html.H3("1. Model Performance Metrics"),
    dcc.Graph(
        id="model-metrics-bar-chart",
        style={"height": "500px"}
    ),

    # Section 2: Confusion Matrix Visualizations
    html.H3("2. Confusion Matrix Visualization"),
    dcc.Dropdown(
        id="confusion-matrix-selector",
        options=[
            {"label": "Logistic Regression", "value": "logistic"},
            {"label": "Random Forest", "value": "rf"},
            {"label": "XGBoost", "value": "xgb"}
        ],
        value="logistic",
        style={"width": "50%"}
    ),
    dcc.Graph(id="confusion-matrix-plot"),

    # Section 3: Data Exploration
    html.H3("3. Data Exploration"),
    dcc.Dropdown(
        id="data-exploration-selector",
        options=[
            {"label": "Histogram: Limit Balance", "value": "hist_limit_bal"},
            {"label": "Scatter Plot: Limit Balance vs Age", "value": "scatter_limit_age"}
        ],
        value="hist_limit_bal",
        style={"width": "50%"}
    ),
    dcc.Graph(id="data-exploration-plot")
])

In [41]:
# Callbacks
@app.callback(
    Output("model-metrics-bar-chart", "figure"),
    Input("model-metrics-bar-chart", "id")
)
def update_model_metrics_bar_chart(_):
    fig = px.bar(
        model_metrics,
        x="Model",
        y=["Accuracy", "Precision", "Recall", "F1-Score", "ROC-AUC"],
        barmode="group",
        title="Model Performance Metrics",
        labels={"value": "Score", "variable": "Metric"}
    )
    return fig


@app.callback(
    Output("confusion-matrix-plot", "figure"),
    Input("confusion-matrix-selector", "value")
)
def update_confusion_matrix_plot(model_type):
    if model_type == "logistic":
        cm = pd.read_csv("C:/Users/user/Documents/PROJECTS\DATA SCIENCE PROJECTS/PROJECTS ON GITHUB/confusion_matrix_logistic.csv")  # Replace with your confusion matrix file
    elif model_type == "rf":
        cm = pd.read_csv("C:/Users/user/Documents/PROJECTS\DATA SCIENCE PROJECTS/PROJECTS ON GITHUB/confusion_matrix_rf.csv")  # Replace with your confusion matrix file
    elif model_type == "xgb":
        cm = pd.read_csv("C:/Users/user/Documents/PROJECTS\DATA SCIENCE PROJECTS/PROJECTS ON GITHUB/confusion_matrix_xgb.csv")  # Replace with your confusion matrix file
    
    fig = px.imshow(
        cm.values,
        labels=dict(x="Predicted", y="Actual"),
        x=["Not Fraud", "Fraud"],
        y=["Not Fraud", "Fraud"],
        text_auto=True,
        title=f"Confusion Matrix: {model_type.capitalize()}"
    )
    return fig


@app.callback(
    Output("data-exploration-plot", "figure"),
    Input("data-exploration-selector", "value")
)
def update_data_exploration_plot(plot_type):
    if plot_type == "hist_limit_bal":
        return px.histogram(data_credit_card, x="LIMIT_BAL", title="Histogram of Limit Balance")
    elif plot_type == "scatter_limit_age":
        return px.scatter(
            data, x="LIMIT_BAL", y="AGE", color="default.payment.next.month",
            title="Limit Balance vs Age (Fraud Highlighted)"
        )

In [42]:
# Run the app in Jupyter
app.run_server(mode="inline", debug=True)

In [13]:
# EDA: Initial Correlation and Descriptions
correlation = data_credit_card.corr()
columns = data_credit_card.columns

# App Layout
app.layout = html.Div([
    html.H1("Fraud Detection Dashboard", style={"text-align": "center"}),

    # Section 1: Data Exploration
    html.H3("1. Exploratory Data Analysis (EDA)"),
    dcc.Dropdown(
        id="eda-plot-selector",
        options=[
            {"label": "Histogram: Limit Balance", "value": "hist_limit_bal"},
            {"label": "Correlation Heatmap", "value": "heatmap"},
            {"label": "Scatter Plot: Limit Balance vs Age", "value": "scatter_limit_age"},
        ],
        value="hist_limit_bal",
        style={"width": "50%"}
    ),
    dcc.Graph(id="eda-plot"),

    # Section 2: Preprocessing
    html.H3("2. Preprocessing Insights"),
    dcc.Dropdown(
        id="preprocessing-view",
        options=[
            {"label": "Original Data Correlation", "value": "original_corr"},
            {"label": "Scaled Data Correlation", "value": "scaled_corr"}
        ],
        value="original_corr",
        style={"width": "50%"}
    ),
    dcc.Graph(id="preprocessing-plot"),

    # Section 3: Model Performance
    html.H3("3. Model Performance"),
    dcc.Dropdown(
        id="model-performance-view",
        options=[
            {"label": "Confusion Matrix", "value": "conf_matrix"},
            {"label": "ROC Curve", "value": "roc_curve"}
        ],
        value="conf_matrix",
        style={"width": "50%"}
    ),
    dcc.Graph(id="model-performance-plot")
])


In [11]:
# EDA Visualization Callback
@app.callback(
    Output("eda-plot", "figure"),
    Input("eda-plot-selector", "value")
)
def update_eda_plot(plot_type):
    if plot_type == "hist_limit_bal":
        return px.histogram(data_credit_card, x="LIMIT_BAL", title="Histogram of Limit Balance")
    elif plot_type == "scatter_limit_age":
        return px.scatter(data_credit_card, x="LIMIT_BAL", y="AGE", color="default.payment.next.month",
                          title="Limit Balance vs Age (Fraud Highlighted)")
    elif plot_type == "heatmap":
        return px.imshow(correlation, title="Correlation Heatmap")

# Preprocessing Visualization Callback
@app.callback(
    Output("preprocessing-plot", "figure"),
    Input("preprocessing-view", "value")
)
def update_preprocessing_plot(view_type):
    if view_type == "original_corr":
        return px.imshow(correlation, title="Original Data Correlation")
    elif view_type == "scaled_corr":
        scaled_data = (data - data.mean()) / data.std()
        scaled_corr = scaled_data.corr()
        return px.imshow(scaled_corr, title="Scaled Data Correlation")

# Model Performance Visualization Callback
@app.callback(
    Output("model-performance-plot", "figure"),
    Input("model-performance-view", "value")
)
def update_model_performance(view_type):
    if view_type == "conf_matrix":
        conf_matrix = pd.read_csv("confusion_matrix.csv")  # Replace with your file
        fig = px.imshow(conf_matrix, title="Confusion Matrix", text_auto=True)
        return fig
    elif view_type == "roc_curve":
        roc_data = pd.read_csv("roc_curve.csv")  # Replace with your file
        fig = px.line(roc_data, x="False Positive Rate", y="True Positive Rate",
                      title="ROC Curve")
        return fig

# Run the app in Jupyter
if __name__ == "__main__":
    app.run_server(mode="inline", debug=True)
5. Run the dashboard:
In your Jupyter notebook or another Python file, import and run the dashboard.
python
CopyEdit
from fraud_detection_dashboard import app

# If you're running it in a Jupyter Notebook
app.run_server(mode="inline", debug=True)


SyntaxError: invalid syntax (2063484475.py, line 47)